# Tutorial #1: Hello world: Develop a feature set and register with managed feature store

Azure ML managed feature store lets you discover, create and operationalize features. Features are the connective tissue in ML lifecycle, starting from prototyping where you experiment with various features to operationalization where models are deployed and feature data is looked up during inference. For information on basics concept of feature store, see [feature store concepts](fs-concepts).

In this tutorial series you will experience how features seamlessly integrates all the phases of ML lifecycle:
Part 1 (this tutorial): Create

This tutorial is the first part of a three part series. In this tutorial you will:
- Create a new minimal feature store resource
- Develop and test featureset locally with feature transformation capability
- Register a feature-store entity with the feature store
- Register the featureset that you developed with the feature store
- Generate sample training data dataframe using the features you created


#### Important

This feature is currently in public preview. This preview version is provided without a service-level agreement, and it's not recommended for production workloads. Certain features might not be supported or might have constrained capabilities. For more information, see [Supplemental Terms of Use for Microsoft Azure Previews](https://azure.microsoft.com/support/legal/preview-supplemental-terms/).

## Prerequisites
Before following the steps in this article, make sure you have the following prerequisites:

* An Azure Machine Learning workspace. If you don't have one, use the steps in the [Quickstart: Create workspace resources](https://learn.microsoft.com/en-us/azure/machine-learning/quickstart-create-resources?view=azureml-api-2) article to create one.
* To perform the steps in this article, your user account must be assigned the owner or contributor role to a resource group where the feature store will be created

## Setup 

#### Prepare the notebook environment for development
Note: This tutorial uses AzureML spark notebook for development. (placeholder: link to ADB document once ready)

1. Clone the examples repository to your local machine: To run the tutorial, first clone the [examples repository (azureml-examples)](https://github.com/azure/azureml-examples)

```bash
git clone --depth 1 https://github.com/Azure/azureml-examples
```

Alternatively you can download a zip file from the [examples repository (azureml-examples)](https://github.com/azure/azureml-examples): click on the `code` dropdown and click `Download ZIP`. Then unzip the contents into a folder in your local machine.

2. Upload the feature store samples directory to project workspace: Open Azure ML studio UI of your Azure ML workspace -> click on "Notebooks" in left nav -> right click on your user name in the directory listing -> click "upload folder" -> select the feature store samples folder from the cloned directory path: `azureml-examples/sdk/python/featurestore-sample`

3. You can either create a new notebook and paste the instructions in this document step by step and execute OR open the existing notebook titled `1.hello_world.ipynb`. You can execute step by step. Keep this document open and refer to it for detailed explanation of the steps. The notebooks are available in the folder: `featurestore_sample/notebooks`. Select either `sdk_only` folder or the `sdk_and_cli` folder. The latter has CLI commands mixed with python sdk useful in ci/cd scenarios.

4. Enable preview access of managed spark: (To be removed in final draft) <BR> 
  (a) click on the "manage preview features" icon (looks like an announcment icon) in the top right nav of this screen <BR>
  (b) enable access by selecting: "Run notebooks and jobs on managed spark"<BR>
  If you have any issues, details steps are [here](https://learn.microsoft.com/en-us/azure/machine-learning/interactive-data-wrangling-with-apache-spark-azure-ml#prerequisites) - just enable the feature is enough for now

5. In the "Compute" dropdown in the top nav, select "AzureML Spark Compute". It may take 1-2 minutes for this activity to complete. Wait for a status bar in the top to display `configure session`

6. Click on "configure session" -> click on "upload conda file" -> select the file `azureml-examples/sdk/python/featurestore-sample/project/env/conda.yml` from your local machine; Also increase the session time out (idle time) if you want to reduce serverless spark cluster startup time.

__Important:__ Except for this step, you need to run all the other steps every time you have a new spark session/session time out


#### Start spark session

In [1]:
# run this cell to start the spark session (any code block will start the session ). This can take around 10 mins.
print("start spark session")

StatementMeta(0d2eca99-c0ff-413b-b338-2fdfc479042b, 139, 6, Finished, Available)

start spark session


#### Setup root directory for the samples

In [2]:
import os

# Please update your alias belpw (or any custom directory you uploaded the samples to).
# You can find the name from the directory structure in the left nav
root_dir = "./Users/<your user alias>/featurestore_sample"

if os.path.isdir(root_dir):
    print("The folder exists.")
else:
    print("The folder does not exist. Please create or fix the path")

StatementMeta(0d2eca99-c0ff-413b-b338-2fdfc479042b, 139, 7, Finished, Available)

## Note
Feature store Vs Project workspace: You will use a featurestore to reuse features across projects. You will use a project workspace(the current workspace) to train and inference models, by leveraging features from feature stores. Many project workspaces can share and reuse a same feature store.

## Note
In this tutorial you will be using two SDK's:

1. Feature store CRUD sdk:  You will use the same SDK, MLClient (package name `azure-ai-ml`), that you use with Azure ML workpace. This will be used for feature store CRUD operations (Create, Update and Delete) for featurestore, featureset and featurestore-entity. This is because feature store is implemented as a type of workspace. 
2. Feature store core sdk: This sdk (`azureml-featurestore`) is meant to be used for feature set development and consumption (you will learn more about these operations later):
- List/Get registered feature set
- Generate/resolve feature retrieval spec
- Execute featureset definition to generate Spark dataframe
- Generate training using a point-in-time join

For this tutorial so you do not need to install any of these explicitly, since the instructions already cover them (conda yaml in the above step include these)

## Step 1: Create a minimal feature store

#### Step 1a: Set feature store parameters
Set name, location and other values for the feature store

In [3]:
# We use the subscription, resource group, region of this active project workspace. 
# You can optionally replace them to create the resources in a different subsciprtion/resourceGroup, or use existing resources
import os

featurestore_name = "my-featurestore"
featurestore_location = "eastus"
featurestore_subscription_id = os.environ["AZUREML_ARM_SUBSCRIPTION"]
featurestore_resource_group_name = os.environ["AZUREML_ARM_RESOURCEGROUP"]


StatementMeta(0d2eca99-c0ff-413b-b338-2fdfc479042b, 139, 8, Finished, Available)

#### Step 1b: Create the feature store

In [4]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    FeatureStore,
    FeatureStoreEntity,
    FeatureSet,
)
from azure.ai.ml.identity import AzureMLOnBehalfOfCredential

ml_client = MLClient(
    AzureMLOnBehalfOfCredential(), 
    subscription_id=featurestore_subscription_id, 
    resource_group_name=featurestore_resource_group_name
)


fs = FeatureStore(
    name=featurestore_name,
    location=featurestore_location
)
# wait for featurestore creation
fs_poller = ml_client.feature_stores.begin_create(fs, update_dependent_resources=True)
print(fs_poller.result())

StatementMeta(0d2eca99-c0ff-413b-b338-2fdfc479042b, 139, 9, Finished, Available)

Class FeatureStore: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ComputeRuntime: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureStoreSettings: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Method feature_stores: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
_AzureMLSparkOnBehalfOfCredential.get_token succeeded
Class ManagedNetwork: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
_AzureMLSparkOnBehalfOfCredential.get_token succeeded
The deployment request fep-fs-pup-test3-2562221 was accepted. ARM deployment URI for reference: 
https://portal.azure.com//#blade/HubsExtensi

application_insights: /subscriptions/1aefdc5e-3a7c-4d71-a9f9-f5d3b03be19a/resourceGroups/fepTestSynapseCost/providers/Microsoft.insights/components/fepfspupinsightsc4f0f676
container_registry: ''
description: fep-fs-pup-test3
display_name: fep-fs-pup-test3
hbi_workspace: false
identity:
  principal_id: 70148dd1-b10b-4d54-b19f-82b076926c5d
  tenant_id: 72f988bf-86f1-41af-91ab-2d7cd011db47
  type: system_assigned
key_vault: /subscriptions/1aefdc5e-3a7c-4d71-a9f9-f5d3b03be19a/resourceGroups/fepTestSynapseCost/providers/Microsoft.Keyvault/vaults/fepfspupkeyvault61a2c4e0
location: eastus2euap
name: fep-fs-pup-test3
public_network_access: Enabled
resource_group: fepTestSynapseCost
storage_account: /subscriptions/1aefdc5e-3a7c-4d71-a9f9-f5d3b03be19a/resourceGroups/fepTestSynapseCost/providers/Microsoft.Storage/storageAccounts/fepfspupstorageeb7b41136
tags:
  createdByToolkit: sdk-v2-1.7.0



#### Step 1c: Initialize AzureML feature store core SDK client
As explained above, this is used to develop and consume features

In [7]:
# feature store client
from azureml.featurestore import FeatureStoreClient
from azure.ai.ml.identity import AzureMLOnBehalfOfCredential

featurestore = FeatureStoreClient(
    credential = AzureMLOnBehalfOfCredential(), 
    subscription_id = featurestore_subscription_id, 
    resource_group_name = featurestore_resource_group_name, 
    name = featurestore_name
)

StatementMeta(0d2eca99-c0ff-413b-b338-2fdfc479042b, 139, 12, Finished, Available)

_AzureMLSparkOnBehalfOfCredential.get_token succeeded


## Step 2: Prototype and develop a transaction rolling aggregation featureset in this notebook

#### Step 2a: Explore the transactions source data

#### Note
The sample data used in this notebook is hosted in a public accessible blob container. It can only be read in Spark via `wasbs` driver. When you create feature sets using your own source data, please host them in adls gen2 account and use `abfss` driver in the data path.  

In [8]:
# remove the "." in the roor directory path as we need to generate absolute path to read from spark
transactions_source_data_path= "wasbs://data@azuremlexampledata.blob.core.windows.net/feature-store-prp/datasources/transactions-source/*.parquet"
transactions_src_df = spark.read.parquet(transactions_source_data_path)

display(transactions_src_df.head(5))
# Note: display(training_df.head(5)) displays the timestamp column in a different format. You can can call transactions_src_df.show() to see correctly formatted value

StatementMeta(0d2eca99-c0ff-413b-b338-2fdfc479042b, 139, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, e1951acb-9f0b-46d6-8e09-43173e099c34)

#### Step 2c: Develop a transactions featureset locally

Featureset specification is a self-contained definition of feature set that can be developed and tested locally.

Lets create the following rolling window aggregate features:
- transactions 3-day count
- transactions amount 3-day sum
- transactions amount 3-day avg
- transactions 7-day count
- transactions amount 7-day sum
- transactions amount 7-day avg

__Action__:
- Inspect the feature transformation code file: `featurestore/featuresets/transactions/spec/transformation_code/transaction_transform.py`. You will see how is the rolling aggregation defined for the features. This is a spark transformer.

To understand the feature set and transformations in more detail, see [feature store concepts](fs-concepts-url-todo) and [transformation concepts](fs-transformation-concepts-todo).

In [13]:
from azureml.featurestore import create_feature_set_spec, FeatureSetSpec
from azureml.featurestore.contracts import DateTimeOffset, FeatureSource, TransformationCode, Column, ColumnType, SourceType,TimestampColumn


transactions_featureset_code_path = root_dir  + "/featurestore/featuresets/transactions/transformation_code"

transactions_featureset_spec = create_feature_set_spec(
    source = FeatureSource(
        type = SourceType.parquet,
        path = "wasbs://data@azuremlexampledata.blob.core.windows.net/feature-store-prp/datasources/transactions-source/*.parquet",
        timestamp_column = TimestampColumn(name = "timestamp"),
        source_delay = DateTimeOffset(days = 0, hours = 0, minutes = 20)
    ),
    transformation_code = TransformationCode(
        path = transactions_featureset_code_path,
        transformer_class = "transaction_transform.TransactionFeatureTransformer"
    ),
    index_columns = [
        Column(name = "accountID", type = ColumnType.string)
    ],
    source_lookback = DateTimeOffset(days = 7, hours = 0, minutes = 0),
    temporal_join_lookback = DateTimeOffset(days = 1, hours = 0, minutes = 0),
    infer_schema = True,
)
# Generate a spark dataframe from the feature set specification
transactions_fset_df=transactions_featureset_spec.to_spark_dataframe()
# display few records
display(transactions_fset_df.head(5))

StatementMeta(0d2eca99-c0ff-413b-b338-2fdfc479042b, 139, 18, Finished, Available)

SynapseWidget(Synapse.DataFrame, 4c591cb6-95e9-4216-aa80-8411cb94de14)

#### Step 2d:  Export as feature set spec
Inorder to register the feature set spec with the feature store, it needs to be saved in a specific format. 
Action: Please inspect the generated `transactions` FeaturesetSpec: Open this file from the file tree to see the spec: `featurestore/featuresets/accounts/spec/FeaturesetSpec.yaml`

Spec contains these important elements:

1. `source`: reference to a storage. In this case a parquet file in a blob storage.
1. `features`: list of features and their datatypes. If you provide transformation code (see Day 2 section), the code has to return a dataframe that maps to the features and datatypes.
1. `index_columns`: the join keys required to access values from the feature set

Learn more about it in the [top level feature store entities document](fs-concepts-todo) and the [feature set spec yaml reference](reference-yaml-featureset-spec.md).

The additional benefit of persisting it is that it can be source controlled.

In [17]:
import os

# create a new folder to dump the feature set spec
transactions_featureset_spec_folder = root_dir  + "/featurestore/featuresets/transactions/spec"

# check if the folder exists, create one if not
if not os.path.exists(transactions_featureset_spec_folder):
    os.makedirs(transactions_featureset_spec_folder)

transactions_featureset_spec.dump(transactions_featureset_spec_folder)

StatementMeta(0d2eca99-c0ff-413b-b338-2fdfc479042b, 139, 22, Finished, Available)

## Step 3: Register a feature-store entity
Entity helps enforce best practice that same join key definitions are used across featuresets which uses the same logical entities. Examples of entities are account entity, customer entity etc. Entities are typically created once and reused across feature-sets. For information on basics concept of feature store, see [feature store concepts](fs-concepts).

#### Step 3a: Initialize the Feature Store CRUD client

As explained in the beginning of this tutorial, MLClient is used for CRUD of assets in feature store. The below code looks up the feature store we created in an earlier step. We cannot reuse the same ml_client used above here because the former is scoped at resource group level, which is a prerequisite for creation of feature store. The below one is scoped at feature store level.
 

In [14]:
# mlclient on feature store
fs_client = MLClient(
    AzureMLOnBehalfOfCredential(), 
    featurestore_subscription_id, 
    featurestore_resource_group_name, 
    featurestore_name
)


StatementMeta(0d2eca99-c0ff-413b-b338-2fdfc479042b, 139, 19, Finished, Available)

#### Step 3b: Register `account` entity with the feature store
Create account entity that has join key `accountID` of `string` type. 

In [15]:
from azure.ai.ml.entities import DataColumn

account_entity_config = FeatureStoreEntity(
    name = "account",
    version = "1",
    index_columns = [
        DataColumn(name="accountID", type = "string")
    ],
    description = "This entity represents user account index key accountID.",
    tags = {"data_typ": "nonPII"}

)

poller = fs_client.feature_store_entities.begin_create_or_update(account_entity_config)
print(poller.result())

StatementMeta(0d2eca99-c0ff-413b-b338-2fdfc479042b, 139, 20, Finished, Available)

Class DataColumn: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureStoreEntity: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Method feature_store_entities: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
_AzureMLSparkOnBehalfOfCredential.get_token succeeded


description: This entity represents user account index key accountID.
index_columns:
- name: accountID
  type: DataColumnType.STRING
name: account
properties: {}
tags:
  data_typ: nonPII
version: '1'



## Step 4: Register the transaction featureset with the featurestore
You register a feature set asset with the feature store so that you can share and reuse with others. You also get managed capabilities like versioning and materialization (we will learn in this tutorial series).

The feature set asset has reference to the feature set spec that you created earlier and additional properties like version and materialization settings.

In [18]:
from azure.ai.ml.entities import FeatureSetSpecification

transaction_fset_config = FeatureSet(
    name = "transactions",
    version = "1",
    description = "7-day and 3-day rolling aggregation of transactions featureset",
    entities = ["azureml:account:1"],
    stage = "Development",
    specification = FeatureSetSpecification(path = transactions_featureset_spec_folder),
    tags = {"data_type": "nonPII"}
)

poller = fs_client.feature_sets.begin_create_or_update(transaction_fset_config)
print(poller.result())

StatementMeta(0d2eca99-c0ff-413b-b338-2fdfc479042b, 139, 23, Finished, Available)

Class FeatureSetSpecification: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureSet: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Method feature_sets: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class Feature: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
_AzureMLSparkOnBehalfOfCredential.get_token succeeded
Uploading spec_local (0.0 MBs): 100%|██████████| 3557/3557 [00:00<00:00, 4942.29it/s]




Readonly attribute name will be ignored in class <class 'azure.ai.ml._restclient.v2023_02_01_preview.models._models_py3.FeaturesetVersion'>
description: 7-day and 3-day rolling aggregation of transactions featureset
entities:
- azureml:account:1
name: transactions
specification:
  path: azureml://subscriptions/1aefdc5e-3a7c-4d71-a9f9-f5d3b03be19a/resourcegroups/fepTestSynapseCost/workspaces/fep-fs-pup-test3/datastores/workspaceblobstore/paths/LocalUpload/7fe8fa299f293a6749337671a936100f/spec_local
stage: Development
tags:
  data_type: nonPII
version: '1'



#### Explore the FeatureStore UI
* Goto the [Azure ML global landing page](https://ml.azure.com/home?flight=FeatureStoresPrPr,FeatureStoresPuPr). (todo PuP: remove flight url)
* Click on `Feature stores` in the left nav
* You will see the list of feature stores that you have access to. Click on the feature store that you created above.

You can see the feature set and entity that you created.

Note: Creating and updating feature store assets are possible only through SDK and CLI. You can use the UI to search/browse the feature store.

## Step 5: Generate a training data dataframe using the registered features

#### Step 5a: Load observation data

We start by exploring the observation data. Observation data is typically the core data used in training and inference data. This is then joined with feature data to create the full training data. Observation data is the data captured during the time of the event: in this case it has core transaction data including transaction id, account id, transaction amount. In this case, since it is for training, it also has the target variable appended (is_fraud).

To learn more core concepts including observation data, see [feature retrieval concepts](feature-retrieval-concept-todo) and [feature store concepts](fs-concepts-link-todo)

In [25]:
observation_data_path= "wasbs://data@azuremlexampledata.blob.core.windows.net/feature-store-prp/observation_data/train/*.parquet"
observation_data_df = spark.read.parquet(observation_data_path)
obs_data_timestamp_column = "timestamp"

display(observation_data_df)
# Note: the timestamp column is displayed in a different format. Optionally, you can can call training_df.show() to see correctly formatted value

StatementMeta(0d2eca99-c0ff-413b-b338-2fdfc479042b, 139, 30, Finished, Available)

SynapseWidget(Synapse.DataFrame, 77b042ec-79e7-4d35-a5b2-36f5f518dc6e)

#### Step 5c: Get the registered featureset and list its features

In [20]:
# look up the featureset by providing name and version
transactions_featureset = featurestore.feature_sets.get("transactions", "1")
# list its features
transactions_featureset.features

StatementMeta(0d2eca99-c0ff-413b-b338-2fdfc479042b, 139, 25, Finished, Available)

_AzureMLSparkOnBehalfOfCredential.get_token succeeded
_AzureMLSparkOnBehalfOfCredential.get_token succeeded


[Feature(Name=transaction_3d_count,Type=ColumnType.long),
 Feature(Name=transaction_amount_3d_sum,Type=ColumnType.double),
 Feature(Name=transaction_amount_3d_avg,Type=ColumnType.double),
 Feature(Name=transaction_7d_count,Type=ColumnType.long),
 Feature(Name=transaction_amount_7d_sum,Type=ColumnType.double),
 Feature(Name=transaction_amount_7d_avg,Type=ColumnType.double)]

In [21]:
# print sample values
display(transactions_featureset.to_spark_dataframe().head(5))

StatementMeta(0d2eca99-c0ff-413b-b338-2fdfc479042b, 139, 26, Finished, Available)

FeatureSet: transactions, version: 1, offline store is not configured for its feature store..
FeatureSet: transactions, version: 1, does not have materialization settings..
FeatureSet: transactions, version: 1 load data from source..


SynapseWidget(Synapse.DataFrame, d114bc23-1d5e-40da-a346-9043692b1468)

#### Step 5d: Select features and generate training data
In this step we will select features that we would like to be part of training data and use the feature store sdk to generate the training data.

In [27]:
from azureml.featurestore import get_offline_features

# you can select features in pythonic way
features = [
    transactions_featureset.get_feature('transaction_amount_7d_sum'),
    transactions_featureset.get_feature('transaction_amount_7d_avg')
]

# you can also specify features in string form: featurestore:featureset:version:feature
more_features = [
    "transactions:1:transaction_3d_count",
    "transactions:1:transaction_amount_3d_avg"
]

more_features = featurestore.resolve_feature_uri(more_features)
features.extend(more_features)

# generate training dataframe by using feature data and observation data
training_df = get_offline_features(features = features, observation_data = observation_data_df, timestamp_column = obs_data_timestamp_column)

# Ignore the message that says feature set is not materialized (materialization is optional). We will enable materialization in the next part of the tutorial.
display(training_df)
# Note: the timestamp column is displayed in a different format. Optionally, you can can call training_df.show() to see correctly formatted value

StatementMeta(0d2eca99-c0ff-413b-b338-2fdfc479042b, 139, 32, Finished, Available)

_AzureMLSparkOnBehalfOfCredential.get_token succeeded
_AzureMLSparkOnBehalfOfCredential.get_token succeeded


FeatureSet: transactions, version: 1, offline store is not configured for its feature store..
FeatureSet: transactions, version: 1, does not have materialization settings..
FeatureSet: transactions, version: 1 load data from source..


SynapseWidget(Synapse.DataFrame, 70afae99-9fa5-41d2-8560-de8055824751)

You can see how the features are appended to the training data using a [point-in-time join](fs-concepts-todo#pitjoin). 

We have reached the end of the tutorial. Now you have your training data using features from feature store. You can either save it to storage for later use, or run model training on it directly.

## Cleanup

Part 4 of the tutorial has instructions deleting the resources

## Next steps
* Part 2 of tutorial: run training pipeline with feature store
* Understand concepts: feature store concepts, feature transformation concepts
* Understand identity and access control for feature store
* View feature store trouble shooting guide
* Reference: YAML reference, feature store SDK